## Libraries import

In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Libraries Imports
import pandas as pd
import numpy as np

## Functions definition

In [2]:
# Read csv
def read_csv(path):
    return pd.read_csv(path)

In [3]:
# Send data to csv
def send_to_csv(df,path):
    return df.to_csv(path)

In [4]:
# Drop columns
def drop_columns(df,columns):
    return df.drop(columns, 1) 

In [5]:
# Aggregate data
def agg_data(df, dimensions, agg_var,agg_opp):
    return df.groupby(dimensions, as_index=False).agg({agg_var:agg_opp})

In [6]:
# Join data
def join_data(df_left,df_right,link_fields,link_type):
    return pd.merge(df_left,df_right,on=link_fields,how=link_type)

## Data import

In [7]:
# Import raw data 
df_temperatures=read_csv("../../Data/Raw_Data/temperatures.csv")
df_bank_holidays=read_csv("../../Data/Raw_Data/bank_holidays.csv")
df_electricity_demand=read_csv("../../Data/Raw_Data/electricity_demand.csv")
df_population=read_csv("../../Data/Raw_Data/population.csv")

In [8]:
# Drop not needed columns
df_temperatures=drop_columns(df_temperatures,['Unnamed: 0'])
df_bank_holidays=drop_columns(df_bank_holidays,['Unnamed: 0'])
df_electricity_demand=drop_columns(df_electricity_demand,['Unnamed: 0'])
df_population=drop_columns(df_population,['Unnamed: 0'])

## Regions: Define region master table so cities and regions can be related

In [9]:
# Define region master table
df_regions=pd.DataFrame()

df_regions['Region']=[x for x in df_population['Region'].unique()]

df_regions['City']=['Sevilla','Zaragoza','Oviedo','Santander','Valladolid','Albacete',
                    'Barcelona','Valencia','Badajoz','Vigo','Madrid','Murcia','Pamplona',
                    'Bilbao','Logroño']

# Save table
send_to_csv(df_regions,"../../Data/Intermediate_Data/regions.csv")

# Print info
df_regions

,Region,City
0,Andalucía,Sevilla
1,Aragón,Zaragoza
2,Asturias,Oviedo
3,Cantabria,Santander
4,Castilla y León,Valladolid
5,Castilla-La Mancha,Albacete
6,Cataluña,Barcelona
7,Comunidad Valenciana,Valencia
8,Extremadura,Badajoz
9,Galicia,Vigo


## Population: Calculate population ratio for each region

In [10]:
# Calculate country population over the years and ratio by region
df_county_population=agg_data(df_population,['Year'],'Population','sum')

df_county_population=df_county_population.rename(columns={'Population':'Total_Population'})

df_population=join_data(df_population,df_county_population,'Year','left')
df_population['Population_Ratio']=df_population['Population']/\
df_population['Total_Population']

# Print info
df_population

,Region,Year,Population,Total_Population,Population_Ratio
0,Andalucía,2021,8472407.0,43869377.0,0.193128
1,Andalucía,2020,8464411.0,43932022.0,0.192671
2,Andalucía,2019,8414240.0,43552095.0,0.193199
3,Andalucía,2018,8384408.0,43294859.0,0.193658
4,Andalucía,2017,8379820.0,43176933.0,0.194081
...,...,...,...,...,...
100,La Rioja,2019,316798.0,43552095.0,0.007274
101,La Rioja,2018,315675.0,43294859.0,0.007291
102,La Rioja,2017,315381.0,43176933.0,0.007304
103,La Rioja,2016,315794.0,43177319.0,0.007314


## Temperature: ponderation by region population

In [11]:
# Add region to temperatures
df_temperatures_region=join_data(df_temperatures,df_regions,'City','left')

# Print info
df_temperatures_region

,Time,Date,Year,Month,Day,Hour,City,Temp,Region
0,2015-01-01 00:00:00,2015-01-01,2015,1,1,0,Albacete,268.948527,Castilla-La Mancha
1,2015-01-01 00:00:00,2015-01-01,2015,1,1,0,Badajoz,272.818933,Extremadura
2,2015-01-01 00:00:00,2015-01-01,2015,1,1,0,Barcelona,276.065316,Cataluña
3,2015-01-01 00:00:00,2015-01-01,2015,1,1,0,Bilbao,272.969665,País Vasco
4,2015-01-01 00:00:00,2015-01-01,2015,1,1,0,Logroño,266.218819,La Rioja
...,...,...,...,...,...,...,...,...,...
920515,2021-12-31 23:00:00,2021-12-31,2021,12,31,23,Sevilla,281.146932,Andalucía
920516,2021-12-31 23:00:00,2021-12-31,2021,12,31,23,Valencia,279.901981,Comunidad Valenciana
920517,2021-12-31 23:00:00,2021-12-31,2021,12,31,23,Valladolid,275.407475,Castilla y León
920518,2021-12-31 23:00:00,2021-12-31,2021,12,31,23,Vigo,278.176621,Galicia


In [12]:
# Add population to temperatures and ponderate temperature by region
df_temperatures_population=join_data(df_temperatures_region,df_population,
                                     ['Region','Year'],'left')

df_temperatures_population['Temp_Ponderation']= df_temperatures_population['Temp']\
*df_temperatures_population['Population_Ratio']

# Print info
df_temperatures_population

,Time,Date,Year,Month,Day,Hour,City,Temp,Region,Population,Total_Population,Population_Ratio,Temp_Ponderation
0,2015-01-01 00:00:00,2015-01-01,2015,1,1,0,Albacete,268.948527,Castilla-La Mancha,2059191.0,43249750.0,0.047612,12.805077
1,2015-01-01 00:00:00,2015-01-01,2015,1,1,0,Badajoz,272.818933,Extremadura,1092997.0,43249750.0,0.025272,6.894613
2,2015-01-01 00:00:00,2015-01-01,2015,1,1,0,Barcelona,276.065316,Cataluña,7508106.0,43249750.0,0.173599,47.924616
3,2015-01-01 00:00:00,2015-01-01,2015,1,1,0,Bilbao,272.969665,País Vasco,2189257.0,43249750.0,0.050619,13.817438
4,2015-01-01 00:00:00,2015-01-01,2015,1,1,0,Logroño,266.218819,La Rioja,317053.0,43249750.0,0.007331,1.951583
...,...,...,...,...,...,...,...,...,...,...,...,...,...
920515,2021-12-31 23:00:00,2021-12-31,2021,12,31,23,Sevilla,281.146932,Andalucía,8472407.0,43869377.0,0.193128,54.297357
920516,2021-12-31 23:00:00,2021-12-31,2021,12,31,23,Valencia,279.901981,Comunidad Valenciana,5058138.0,43869377.0,0.115300,32.272691
920517,2021-12-31 23:00:00,2021-12-31,2021,12,31,23,Valladolid,275.407475,Castilla y León,2383139.0,43869377.0,0.054324,14.961104
920518,2021-12-31 23:00:00,2021-12-31,2021,12,31,23,Vigo,278.176621,Galicia,2695645.0,43869377.0,0.061447,17.093140


## Bank holidays: ponderation by region

In [13]:
# Add city to bank holidays
df_bank_holidays_city=join_data(df_bank_holidays,df_regions,['Region'],'left')

df_bank_holidays_city['City']=df_bank_holidays_city['City'].fillna('Nacional')

def national_holiday(row):
    if row=='Nacional':
        return 1
    else:
        return 0
    
df_bank_holidays_city['Country_Bank_Holiday']=df_bank_holidays_city['Region']\
.apply(lambda row: national_holiday(row))

# Print info
df_bank_holidays_city    

,Date,Year,Month,Day,Region,City,Country_Bank_Holiday
0,2015-01-01,2015,1,1,Nacional,Nacional,1
1,2015-01-06,2015,1,6,Nacional,Nacional,1
2,2015-02-28,2015,2,28,Andalucía,Sevilla,0
3,2015-03-19,2015,3,19,Comunidad Valenciana,Valencia,0
4,2015-03-19,2015,3,19,Madrid,Madrid,0
...,...,...,...,...,...,...,...
398,2021-10-09,2021,10,9,Comunidad Valenciana,Valencia,0
399,2021-10-12,2021,10,12,Nacional,Nacional,1
400,2021-11-01,2021,11,1,Nacional,Nacional,1
401,2021-12-06,2021,12,6,Nacional,Nacional,1


In [14]:
# Take population and calculate bank holiday ratio
df_bank_holidays_population=\
join_data(df_bank_holidays_city,df_population,['Region','Year'],'left')

df_bank_holidays_population['Population_Ratio']=\
df_bank_holidays_population['Population_Ratio'].fillna(1)

df_bank_holidays_population = \
drop_columns(df_bank_holidays_population,['Population','Total_Population'])

df_bank_holidays_population['Partial_Bank_Holiday'] = \
np.where(df_bank_holidays_population['City']!='Nacional',1,0)

df_bank_holidays_population['Partial_Bank_Holiday_Weight']=\
df_bank_holidays_population['Partial_Bank_Holiday']*\
df_bank_holidays_population['Population_Ratio']

# Print info
df_bank_holidays_population

,Date,Year,Month,Day,Region,City,Country_Bank_Holiday,Population_Ratio,Partial_Bank_Holiday,Partial_Bank_Holiday_Weight
0,2015-01-01,2015,1,1,Nacional,Nacional,1,1.000000,0,0.000000
1,2015-01-06,2015,1,6,Nacional,Nacional,1,1.000000,0,0.000000
2,2015-02-28,2015,2,28,Andalucía,Sevilla,0,0.194199,1,0.194199
3,2015-03-19,2015,3,19,Comunidad Valenciana,Valencia,0,0.115161,1,0.115161
4,2015-03-19,2015,3,19,Madrid,Madrid,0,0.148833,1,0.148833
...,...,...,...,...,...,...,...,...,...,...
398,2021-10-09,2021,10,9,Comunidad Valenciana,Valencia,0,0.115300,1,0.115300
399,2021-10-12,2021,10,12,Nacional,Nacional,1,1.000000,0,0.000000
400,2021-11-01,2021,11,1,Nacional,Nacional,1,1.000000,0,0.000000
401,2021-12-06,2021,12,6,Nacional,Nacional,1,1.000000,0,0.000000


In [15]:
# Aggregate data by day
df_bank_holidays_agg=df_bank_holidays_population.groupby(['Date','Year','Month','Day'], \
as_index=False).agg(Country_Bank_Holiday=('Country_Bank_Holiday', 'mean'), \
Partial_Bank_Holiday=('Partial_Bank_Holiday', 'mean'),\
Partial_Bank_Holiday_Weight=('Partial_Bank_Holiday_Weight','sum'))

# Save table
send_to_csv(df_bank_holidays_agg,"../../Data/Intermediate_Data/bank_holidays_agg.csv")

# Print info
df_bank_holidays_agg

,Date,Year,Month,Day,Country_Bank_Holiday,Partial_Bank_Holiday,Partial_Bank_Holiday_Weight
0,2015-01-01,2015,1,1,1.0,0.0,0.000000
1,2015-01-06,2015,1,6,1.0,0.0,0.000000
2,2015-02-28,2015,2,28,0.0,1.0,0.194199
3,2015-03-19,2015,3,19,0.0,1.0,0.363348
4,2015-03-20,2015,3,20,0.0,1.0,0.063176
...,...,...,...,...,...,...,...
161,2021-10-09,2021,10,9,0.0,1.0,0.115300
162,2021-10-12,2021,10,12,1.0,0.0,0.000000
163,2021-11-01,2021,11,1,1.0,0.0,0.000000
164,2021-12-06,2021,12,6,1.0,0.0,0.000000


## Temperature: aggregation by country

In [16]:
# Temperature aggregation for the whole country
df_county_temp=agg_data(df_temperatures_population, 
                        ['Time','Date','Year','Month','Day','Hour'],'Temp_Ponderation','sum')

# save table
send_to_csv(df_county_temp,"../../Data/Intermediate_Data/county_temp.csv")

# Print info
df_county_temp

,Time,Date,Year,Month,Day,Hour,Temp_Ponderation
0,2015-01-01 00:00:00,2015-01-01,2015,1,1,0,272.368163
1,2015-01-01 01:00:00,2015-01-01,2015,1,1,1,272.047456
2,2015-01-01 02:00:00,2015-01-01,2015,1,1,2,271.796548
3,2015-01-01 03:00:00,2015-01-01,2015,1,1,3,271.602937
4,2015-01-01 04:00:00,2015-01-01,2015,1,1,4,271.459464
...,...,...,...,...,...,...,...
61363,2021-12-31 19:00:00,2021-12-31,2021,12,31,19,281.005748
61364,2021-12-31 20:00:00,2021-12-31,2021,12,31,20,280.474065
61365,2021-12-31 21:00:00,2021-12-31,2021,12,31,21,279.770309
61366,2021-12-31 22:00:00,2021-12-31,2021,12,31,22,279.171545


## Temperature: Add bank holidays

In [17]:
# Add bank holidays to temperature table
df_county_temp_holidays=join_data(df_county_temp,df_bank_holidays_agg,
                                  ['Date','Year','Month','Day'],'left')

df_county_temp_holidays['Country_Bank_Holiday']=\
df_county_temp_holidays['Country_Bank_Holiday'].fillna(0)

df_county_temp_holidays['Partial_Bank_Holiday']=\
df_county_temp_holidays['Partial_Bank_Holiday'].fillna(0)

df_county_temp_holidays['Partial_Bank_Holiday_Weight']=\
df_county_temp_holidays['Partial_Bank_Holiday_Weight'].fillna(0)

# save table
send_to_csv(df_county_temp_holidays,"../../Data/Intermediate_Data/county_temp_holidays.csv")

# Print info
df_county_temp_holidays

,Time,Date,Year,Month,Day,Hour,Temp_Ponderation,Country_Bank_Holiday,Partial_Bank_Holiday,Partial_Bank_Holiday_Weight
0,2015-01-01 00:00:00,2015-01-01,2015,1,1,0,272.368163,1.0,0.0,0.0
1,2015-01-01 01:00:00,2015-01-01,2015,1,1,1,272.047456,1.0,0.0,0.0
2,2015-01-01 02:00:00,2015-01-01,2015,1,1,2,271.796548,1.0,0.0,0.0
3,2015-01-01 03:00:00,2015-01-01,2015,1,1,3,271.602937,1.0,0.0,0.0
4,2015-01-01 04:00:00,2015-01-01,2015,1,1,4,271.459464,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
61363,2021-12-31 19:00:00,2021-12-31,2021,12,31,19,281.005748,0.0,0.0,0.0
61364,2021-12-31 20:00:00,2021-12-31,2021,12,31,20,280.474065,0.0,0.0,0.0
61365,2021-12-31 21:00:00,2021-12-31,2021,12,31,21,279.770309,0.0,0.0,0.0
61366,2021-12-31 22:00:00,2021-12-31,2021,12,31,22,279.171545,0.0,0.0,0.0


## Final table: Merge demand and temperature

In [18]:
# Merge demand and temperatute
df_county_temp_demand=join_data(df_electricity_demand,df_county_temp_holidays,
          ['Time','Date','Year','Month','Day','Hour'],'inner')
df_county_temp_demand=drop_columns(df_county_temp_demand,['utcDateTime'])

# Add population
df_country_population=agg_data(df_population, ['Year'], 'Population','sum')

df_electricity_demand=join_data(df_county_temp_demand,df_country_population,'Year','left')

# Change data types
df_electricity_demand['DemandaElect_ES_MWh']=\
df_electricity_demand['DemandaElect_ES_MWh'].str.replace(',', '.').astype(float)

# Rename columns
df_electricity_demand.rename(columns=\
{'DemandaElect_ES_MWh': 'Demand_MWh', 'Temp_Ponderation': 'Temp_K'},inplace=True)

# Save table
send_to_csv(df_electricity_demand,"../../Data/Intermediate_Data/electricity_demand.csv")

# Print info
df_electricity_demand

,Time,Date,Year,Month,Day,Hour,Demand_MWh,Temp_K,Country_Bank_Holiday,Partial_Bank_Holiday,Partial_Bank_Holiday_Weight,Population
0,2015-01-01 00:00:00,2015-01-01,2015,1,1,0,24511.5000,272.368163,1.0,0.0,0.0,43249750.0
1,2015-01-01 01:00:00,2015-01-01,2015,1,1,1,22866.1667,272.047456,1.0,0.0,0.0,43249750.0
2,2015-01-01 02:00:00,2015-01-01,2015,1,1,2,21392.8333,271.796548,1.0,0.0,0.0,43249750.0
3,2015-01-01 03:00:00,2015-01-01,2015,1,1,3,20319.6667,271.602937,1.0,0.0,0.0,43249750.0
4,2015-01-01 04:00:00,2015-01-01,2015,1,1,4,19923.0000,271.459464,1.0,0.0,0.0,43249750.0
...,...,...,...,...,...,...,...,...,...,...,...,...
61363,2021-12-31 19:00:00,2021-12-31,2021,12,31,19,27653.1667,281.005748,0.0,0.0,0.0,43869377.0
61364,2021-12-31 20:00:00,2021-12-31,2021,12,31,20,26746.5000,280.474065,0.0,0.0,0.0,43869377.0
61365,2021-12-31 21:00:00,2021-12-31,2021,12,31,21,23952.6667,279.770309,0.0,0.0,0.0,43869377.0
61366,2021-12-31 22:00:00,2021-12-31,2021,12,31,22,22324.8333,279.171545,0.0,0.0,0.0,43869377.0
